# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/city_data_clean.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hong Kong,22.29,114.16,90.00,74,95,11.99,HK,07/18/20
1,Mataura,-46.19,168.86,39.00,83,0,3.00,NZ,07/18/20
2,Lensk,60.73,114.93,59.31,71,98,9.66,RU,07/18/20
3,Saint George,37.10,-113.58,102.99,12,1,17.22,US,07/18/20
4,Yaan,7.38,8.57,70.34,96,100,4.41,NG,07/18/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]].astype(float)
#cities_df = cities_df.dropna()
humidity = cities_df['Humidity'].astype(float)

In [4]:
#Create a humidity heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                     max_intensity=100, point_radius=3)

fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
index = cities_df.loc[(cities_df['Max Temp'] < 75) |
                      (cities_df['Max Temp'] > 85) |
                      (cities_df['Wind Speed'] > 10) |
                      (cities_df['Cloudiness'] > 0)].index
select_cities_df = cities_df.drop(index, inplace=False)
select_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Reguengos de Monsaraz,38.43,-7.53,81.00,53,0,4.70,PT,07/18/20
35,Albany,42.60,-73.97,78.01,61,0,1.01,US,07/18/20
108,Sirte,31.21,16.59,76.53,80,0,8.25,LY,07/18/20
114,Lázaro Cárdenas,17.96,-102.20,83.59,76,0,6.55,MX,07/18/20
125,Syracuse,37.09,15.28,75.00,78,0,2.24,IT,07/18/20


In [6]:
select_cities_df = select_cities_df.dropna(inplace=False)
select_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Reguengos de Monsaraz,38.43,-7.53,81.00,53,0,4.70,PT,07/18/20
35,Albany,42.60,-73.97,78.01,61,0,1.01,US,07/18/20
108,Sirte,31.21,16.59,76.53,80,0,8.25,LY,07/18/20
114,Lázaro Cárdenas,17.96,-102.20,83.59,76,0,6.55,MX,07/18/20
125,Syracuse,37.09,15.28,75.00,78,0,2.24,IT,07/18/20
265,Sakakah,29.97,40.21,84.20,12,0,3.36,SA,07/18/20
270,Termez,37.22,67.28,77.00,29,0,4.70,UZ,07/18/20
325,Ternate,0.80,127.40,82.11,74,0,2.35,ID,07/18/20
347,Jalu,29.03,21.55,78.48,53,0,8.30,LY,07/18/20
393,Jacareacanga,-6.22,-57.75,77.25,68,0,1.77,BR,07/18/20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame()
hotel_df["Hotel Name"] = []
hotel_df

,Hotel Name


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
cities = []
countries = []

for index, row in select_cities_df.iterrows():
    location = f"{row['Lat']},{row['Lng']}"
    radius = 5000
    search_type = "hotels"
    
    params = {
        "location" : location,
        "radius" : radius,
        "type" : search_type,
        "key" : g_key
    }
    
    try:
        print(f"Searching near {row['City']}, {row['Country']}...")
        response = requests.get(base_url, params=params)
        places = response.json()

        name = places['results'][0]['name']
        
        hotels.append(name)
        cities.append(row["City"])
        countries.append(row["Country"])
                
    except KeyError:
        print(f"No hotels found within 5000m of {row['City']}, {row['Country']}!")
        
hotel_df["Hotel Name"] = hotels
hotel_df["City"] = cities
hotel_df["Country"] = countries

Searching near Reguengos de Monsaraz, PT...
Searching near Albany, US...
Searching near Sirte, LY...
Searching near Lázaro Cárdenas, MX...
Searching near Syracuse, IT...
Searching near Sakakah, SA...
Searching near Termez, UZ...
Searching near Ternate, ID...
Searching near Jalu, LY...
Searching near Jacareacanga, BR...
Searching near Chālūs, IR...
Searching near Baishishan, CN...
Searching near Zhangaözen, KZ...
Searching near Tucumã, BR...
Searching near Ust-Maya, RU...
Searching near Shīeli, KZ...
Searching near Dwārka, IN...
Searching near Emba, KZ...
Searching near Yakovlevka, RU...
Searching near La Santisima Trinidad, BO...
Searching near Asunción, PY...


In [9]:
hotel_df

,Hotel Name,City,Country
0,Reguengos de Monsaraz,Reguengos de Monsaraz,PT
1,Voorheesville,Albany,US
2,Sirte,Sirte,LY
3,Lázaro Cárdenas,Lázaro Cárdenas,MX
4,Syracuse,Syracuse,IT
5,Sakaka,Sakakah,SA
6,Termez,Termez,UZ
7,Ternate,Ternate,ID
8,Jalu,Jalu,LY
9,Jacareacanga,Jacareacanga,BR


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = select_cities_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))